In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
gaon_df = pd.read_csv('../data/gaon_final_data.csv', index_col=0)
melon_df = pd.read_csv('../data/melon_final_data.csv', index_col=0)

### 1.데이터 특수문자 공백제거

In [3]:
# test1 = '더보이즈 (the boyz)'
# # test2 = 'stayc (스테이씨)'
# # test3 = '아이브'

# if test1.find('(') != -1:
#     p = re.compile('\(([^)]+)')
#     data_bracket = p.findall(test1)[0]
#     test1 = re.sub('\([^)]*\)','', test1)

In [4]:
# 괄호 따로 저장해서 컬럼으로 추가
### 가수 괄호는 따로 컬럼에 저장 *혹시 데이터 일치여부에 사용할 수 있으니

def Save_Bracket(df, column):
    result = []
    new_column = []
    for i in range(len(df)):
        data = df.iloc[i][f'{column}']

        # 소문자로 바꾸기
        data = data.lower()

        # 괄호 안 내용 저장
        if data.find('(') != -1:
            p = re.compile('\(([^)]+)')
            data_bracket = p.findall(data)[0]
            # 괄호 삭제
            data = re.sub('\([^)]*\)','', data)
            # 양 쪽 공백 제거
            data = data.strip()

            # 리스트에 데이터 추가
            result.append(data)
            new_column.append(data_bracket)
            
        else:
            result.append(data)
            new_column.append(np.NaN)

    df[f'{column}'] = result
    df[f'{column} bracket'] = new_column

    return df

# 멜론 데이터 전처리
def Make_Clean_Name(df, column):
    result = []
    for i in range(len(df)):
        data = df.iloc[i][f'{column}']

        # 특수문자 제거 (콤마는 빼고 제거)
        data = re.sub('[-=+#/\?:^.@★*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', " ",data)

        # 소문자로 바꾸기
        data = data.lower()
        
        if column == 'singer':
            # 가수이름은 모든 공백 제거
            data = data.replace(' ','')
        else:  
            # 앞,뒤 공백 제거
            data = data.strip()

        result.append(data)

    df[f'{column}'] = result

    return df


In [5]:
# gaon_df 전처리
gaon_df = Save_Bracket(gaon_df, 'music')
gaon_df = Save_Bracket(gaon_df, 'singer')
gaon_df = Make_Clean_Name(gaon_df, 'music')
gaon_df = Make_Clean_Name(gaon_df, 'singer')

# melon_df 전처리
melon_df = Make_Clean_Name(melon_df, 'music')
melon_df = Make_Clean_Name(melon_df, 'singer')

### 2.차트에 든 횟수 DV(종속변수) 컬럼 만들기

In [6]:
gaon_df['singer-music'] = gaon_df['singer'] + '-' + gaon_df['music']
DV_df = pd.DataFrame(gaon_df['singer-music'].value_counts())
DV_df.reset_index(drop=False, inplace=True)
DV_df.rename(columns={'index':'singer-music', 'singer-music':'rank_count'}, inplace=True)

In [7]:
DV_df

,singer-music,rank_count
0,방탄소년단-봄날,324
1,"폴킴-모든 날, 모든 순간",268
2,"akmu-어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",171
3,아이유-밤편지,169
4,임영웅-이제 나만 믿어요,161
...,...,...
7369,"백지영,나원주-니가 내리는 날",1
7370,에이핑크-동화 같은 사랑,1
7371,에이핑크-천사가 아냐,1
7372,싸이-rock will never die,1


### 3.멜론에 가온 start_date 붙이기

In [8]:
def Add_startdate(df1, df2):
    date_list = []
    for i in range(len(df1)):

        music = df1['music'].iloc[i]
        singer = df1['singer'].iloc[i]

        try:
            start_date = df2['start_date'].loc[(df2['music']==music) & ((df2['singer']==singer)|(df2['singer bracket']==singer))].iloc[0]
            date_list.append(start_date)
        except:
            date_list.append(np.NaN)

    df1['start_date'] = date_list

    return df1

In [9]:
melon_df = Add_startdate(melon_df, gaon_df)

#### start,release로 7일이내 순위진입 T,F 뽑아서 컬럼

In [10]:
# 날짜 datetime 형식으로 변환환
melon_df['start_date'] = pd.to_datetime(melon_df['start_date'])
melon_df['release_date'] = pd.to_datetime(melon_df['release_date'])

# 7일 계산해서 calculate컬럼으로 추가
melon_df['calculate'] = melon_df['release_date'] - melon_df['start_date']
# 7일이내면 값 1, 아니면 0
melon_df['within 7days'] =  abs(melon_df['calculate']) < pd.Timedelta(days=7)
melon_df['within 7days'].replace({True:1, False:0}, inplace=True)

# 쓸모없는 컬럼 삭제
melon_df.drop(columns=['start_date', 'calculate'], inplace=True)

### 4.melon에 카테고리에서 국내드라마로 OST 컬럼 이진변수 만들기

In [11]:
# ost컬럼 추가 + 카테고리에서 발라드,국내드라마에서 국내드라마는 삭제
def Make_OST_column(df):
    ost_list = []
    category_list = []

    for i in range(len(df)):
        category = df['category'].iloc[i]
        try:
            a, b = category.split(', ')
            if b == '국내드라마':
                ost_list.append(1)
                category_list.append(a)
            else:
                ost_list.append(0)
                category_list.append(category)
        except:
            ost_list.append(0)
            category_list.append(category)

    df['ost'] = ost_list
    df['category'] = category_list

    return df

In [12]:
melon_df = Make_OST_column(melon_df)

In [13]:
melon_df['ost'].value_counts()

0    4447
1     459
Name: ost, dtype: int64

In [14]:
# 고유값인 가수-노래제목 컬럼 만들기
melon_df['singer-music'] = melon_df['singer']+'-'+melon_df['music']

# 가수-노래제목 기준으로 중복 제거
melon_df.drop_duplicates(subset='singer-music', inplace=True)

# 인덱스 재정렬
melon_df.reset_index(drop=True, inplace=True)

### 5. 종속변수(정답값) 붙이기 - 가온 차트인 횟수

In [15]:
# 가수 괄호 딕셔너리 만들기
singer_bracket_dict = {}
bracket_df = gaon_df[gaon_df['singer bracket'].notnull()]
bracket_df = bracket_df[['singer', 'singer bracket']]
bracket_dict = bracket_df.set_index('singer').T.to_dict('records')
bracket_dict = bracket_dict[0]

C:\Users\bin\AppData\Local\Temp\ipykernel_6740\2359084250.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  bracket_dict = bracket_df.set_index('singer').T.to_dict('records')


In [16]:
bracket_dict['케이윌']

'k.will'

In [17]:
# 종속변수 붙이기

def Make_DV(df1, df2, dict):
    
    count_list = []
    for i in range(len(df1)):

        # 멜론 데이터에 있는 노래가 DV_1에 있는 지 확인 후, rank_count 데이터 붙이기
        singer_music = df1['singer-music'].iloc[i]
        singer = df1['singer'].iloc[i]
        music = df1['music'].iloc[i]

        try:
            # 괄호 안의 이름으로도 찾아보기
            singer_bracket = dict[singer]
            global singer_singerbracket
            singer_singerbracket = singer_bracket+'-'+music
            

            # DV_df에 해당 노래가 있으면, rank_count값을 리스트에 추가
            if len(df2.loc[(df2['singer-music']==singer_music)|(df2['singer-music']==singer_singerbracket)]) > 0:
                rank_count = df2['rank_count'].loc[(df2['singer-music']==singer_music)|(df2['singer-music']==singer_singerbracket)].iloc[0]
                count_list.append(rank_count)
            else:
                count_list.append(np.NaN)
        except:
            # DV_df에 해당 노래가 있으면, rank_count값을 리스트에 추가
            if len(df2.loc[(df2['singer-music']==singer_music)]) > 0:
                rank_count = df2['rank_count'].loc[df2['singer-music']==singer_music].iloc[0]
                count_list.append(rank_count)
            else:
                count_list.append(np.NaN)

    df1['rank_count'] = count_list

    return df1

In [18]:
melon_df = Make_DV(melon_df, DV_df, bracket_dict)

In [19]:
# # 종속변수 붙이기

# count_list = []
# for i in range(len(melon_df)):

#     # 멜론 데이터에 있는 노래가 DV_df에 있는 지 확인 후, rank_count 데이터 붙이기
#     singer_music = melon_df['singer-music'].iloc[i]
#     singer = melon_df['singer'].iloc[i]
#     music = melon_df['music'].iloc[i]

#     try:
#         # 괄호 안의 이름으로도 찾아보기
#         singer_bracket = bracket_dict[singer]
#         singer_singerbracket = singer_bracket+'-'+music

#         # DV_df에 해당 노래가 있으면, rank_count값을 리스트에 추가
#         if len(DV_df.loc[(DV_df['singer-music']==singer_music)|(DV_df['singer-music']==singer_singerbracket)]) > 0:
#             rank_count = DV_df['rank_count'].loc[(DV_df['singer-music']==singer_music)|(DV_df['singer-music']==singer_singerbracket)].iloc[0]
#             count_list.append(rank_count)
#         else:
#             count_list.append(np.NaN)
#     except:
#         # DV_df에 해당 노래가 있으면, rank_count값을 리스트에 추가
#         if len(DV_df.loc[(DV_df['singer-music']==singer_music)|(DV_df['singer-music']==singer_singerbracket)]) > 0:
#             rank_count = DV_df['rank_count'].loc[DV_df['singer-music']==singer_music].iloc[0]
#             count_list.append(rank_count)
#         else:
#             count_list.append(np.NaN)
# melon_df['rank_count'] = count_list

In [20]:
# 필요없는 컬럼 singer-music 삭제
melon_df.drop(columns=['singer-music'], inplace=True, axis=0)

In [21]:
melon_df

,music,singer,music_id,release_date,category,music_like,comment_count,within 7days,ost,rank_count
0,dancing queen,소녀시대,3994911,2013-01-01,댄스,21566,36,1,0,11
1,사랑은 이렇게,케이윌,3995922,2012-12-22,발라드,9927,11,0,1,8
2,숲,이승기,3968836,2012-11-22,발라드,155630,361,0,0,3
3,카페인,양요섭,3971021,2012-11-26,발라드,80677,205,0,0,14
4,매력있어,akmu,3987723,2012-12-12,포크/블루스,38280,36,0,0,17
...,...,...,...,...,...,...,...,...,...,...
4380,나의 바람,비투비,36414423,2023-05-02,댄스,29084,776,1,0,1
4381,heaven,임재현,36394038,2023-04-25,발라드,16137,41,1,1,1
4382,안녕이라고 말하지마,vos,36410735,2023-04-30,발라드,7537,22,1,0,1
4383,"이브, 프시케 그리고 푸른 수염의 아내",lesserafim,36411344,2023-05-01,댄스,14849,264,1,0,1


In [22]:
melon_df.to_csv('./melon_pre.csv')

### 6.가수 이름 word2vec 

In [20]:
from gensim.models.word2vec import Word2Vec

In [21]:
m_singer_list = melon_df['singer'].tolist()

In [22]:
#모델 불러오기
#리스트인 melon_singer를 word2vec , mincount 1로 1개이상 값은 다 

model = Word2Vec(sentences=[m_singer_list], min_count=1)

singer_w2_dict = {}
for singer in m_singer_list:
    singer_w2_dict[singer] = model.wv[singer]


In [23]:
# singer값을 word2vec 데이터로 바꾸기

def make_word2vec_column(df,column,dict):
    w2_list = []
    for i in range(len(melon_df)):

        singer = melon_df[f'{column}'].iloc[i]
        try:
            data = dict[singer]
            w2_list.append(data)
        except:
            w2_list.append(np.NaN)

    df[f'{column}'] = w2_list

    return df

In [24]:
melon = make_word2vec_column(melon_df,'singer',singer_w2_dict)

### 7.melon 카테고리에서 자른것들 원핫인코딩

In [126]:
# # 카테고리 세분화 - 2개까지만
# def Divide_category(df):
#     cate_1 = []
#     cate_2 = []


#     for i in range(len(df)):
#         category = df['category'].iloc[i]
#         category_list = category.split(',')
#         if len(category_list) == 1:
#             cate_1.append(category_list[0])
#             cate_2.append(np.NaN)
#         elif len(category_list) >= 2:
#             cate_1.append(category_list[0])
#             cate_2.append(category_list[1])
        
#     df['category_1'] = cate_1
#     df['category_2'] = cate_2

#     return df

# 카테고리 맨 앞꺼 빼고 삭제하는 함수
def delete_category(df):
    cate = []

    for i in range(len(df)):
        category = df['category'].iloc[i]
        category_list = category.split(',')
        cate.append(category_list[0])

    df['category'] = cate

    return df

In [127]:
melon_df = delete_category(melon_df)

In [128]:
from sklearn.preprocessing import OneHotEncoder

In [129]:
ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(melon_df[['category']])
melon_df = pd.concat([melon_df.drop(columns=['category']),
           pd.DataFrame(train_cat, columns=['category_' + col for col in ohe.categories_[0]])], axis=1)

c:\Users\bin\anaconda3\envs\shinhanAI\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [131]:
melon_df.columns

Index(['music', 'singer', 'music_id', 'release_date', 'music_like',
       'comment_count', 'within 7days', 'ost', 'singer-music', 'rank_count',
       'category_CCM', 'category_J-POP', 'category_POP', 'category_R&B/Soul',
       'category_게임', 'category_국내드라마', 'category_국내영화', 'category_국외영화',
       'category_댄스', 'category_랩/힙합', 'category_록/메탈', 'category_발라드',
       'category_성인가요/트로트', 'category_애니메이션/웹툰', 'category_월드뮤직',
       'category_인디음악', 'category_일렉트로니카', 'category_재즈', 'category_클래식',
       'category_포크/블루스'],
      dtype='object')

In [132]:
melon_df.to_csv('./music_pre.csv')

In [9]:
import pandas as pd
df = pd.read_csv('./music_pre.csv', index_col=0)

In [11]:
df.columns

Index(['music', 'singer', 'music_id', 'release_date', 'music_like',
       'comment_count', 'within 7days', 'ost', 'rank_count', 'category_CCM',
       'category_J-POP', 'category_POP', 'category_R&B/Soul', 'category_게임',
       'category_국내드라마', 'category_국내영화', 'category_국외영화', 'category_댄스',
       'category_랩/힙합', 'category_록/메탈', 'category_발라드', 'category_성인가요/트로트',
       'category_애니메이션/웹툰', 'category_월드뮤직', 'category_인디음악',
       'category_일렉트로니카', 'category_재즈', 'category_클래식', 'category_포크/블루스'],
      dtype='object')